In [ ]:
from problema.problema import Problema
import pulp as pu
import pandas as pd
import plotly.graph_objects as go

In [ ]:
problema = Problema(file='model_template_1205v2.xlsm')

In [ ]:
problema.solve(t_limit_minutes=10)

In [ ]:
transporte_df = problema.reporte_transporte()
reporte_plantas_df = problema.reporte_planta()
puerto_df = problema.reporte_puerto()

In [ ]:
with pd.ExcelWriter(path='model_template_1205v2_output.xlsx') as writer:
    transporte_df.to_excel(writer, sheet_name='transporte', index=False)
    reporte_plantas_df.to_excel(writer, sheet_name='plantas', index=False)
    puerto_df.to_excel(writer, sheet_name='Puerto', index=False)


In [ ]:
def dibujar_inventario_ingrediente_puerto(df: pd.DataFrame, puerto:str, ingrediente: str) -> go.Figure:
    fig = go.Figure()

    temp = df[(df['ingrediente'] == ingrediente) & (df['puerto'] == puerto)].copy()

    temp['key'] = temp['empresa'] + ", " + temp['importacion']

    temp = temp.pivot_table(index='fecha', columns='key', values='inventario_kg', aggfunc='sum').reset_index()

    for importacion in temp.drop(columns=['fecha']).columns:
        fig.add_trace(go.Scatter(
            x=temp['fecha'],  # x-axis
            y=temp[importacion],  # y-axis
            mode='lines',  # Connect data points with lines
            name=importacion  # Name in the legend
        )
        )

    # Layout parameters
    fig.update_layout(
        title=f'Puerto: {puerto}, Ingrediente: {ingrediente}',  # Title
        xaxis_title='fecha',  # y-axis name
        yaxis_title='Kg',  # x-axis name
        xaxis_tickangle=0,  # Set the x-axis label angle
        showlegend=True,     # Display the legend
        barmode='stack',
    )
    return fig

In [ ]:
for puerto in puerto_df['puerto'].unique():
    fig = dibujar_inventario_ingrediente_puerto(df=puerto_df, puerto=puerto, ingrediente='maiz')
    fig.show()

In [ ]:
def dibujar_inventario_puerto(df: pd.DataFrame, empresa:str, puerto:str, ingrediente:str, importacion:str) -> go.Figure:
    
    df['despachos_indirectos_kg'] = 34000*df['despachos_indirectos']

    temp = df[(df['empresa'] == empresa) &
              (df['ingrediente'] == ingrediente) &
              (df['importacion']==importacion)].copy()


    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=temp['fecha'],  # x-axis
        y=temp['inventario_kg'],  # y-axis
        mode='lines',  # Connect data points with lines
        name='Inventario'  # Name in the legend
    )
    )

    fig.add_trace(go.Bar(
        x=temp['fecha'], 
        y=temp['ingreso_bodega'],
        name='Ingreso a Bodega')
    )

    fig.add_trace(go.Bar(
        x=temp['fecha'], 
        y=temp['despachos_indirectos_kg'],
        name='Despacho hacia planta',
        hovertext=temp['plantas'])
    )

    # Layout parameters
    fig.update_layout(
        title=f'Puerto: {puerto}, Empresa: {empresa}, Ingrediente: {ingrediente}, Importacion: {importacion}',  # Title
        xaxis_title='Fecha',  # y-axis name
        yaxis_title='Kg',  # x-axis name
        xaxis_tickangle=0,  # Set the x-axis label angle
        showlegend=True,     # Display the legend
        barmode='stack',
        legend=dict(orientation='h',  x=0.5, xanchor='center'),
    )

    return fig

In [ ]:
grupo_puerto = puerto_df.groupby(by=['puerto', 'empresa', 'importacion', 'ingrediente'])[
    ['inventario_kg']].sum().reset_index()
grupo_puerto = grupo_puerto[grupo_puerto['inventario_kg'] > 0]
grupo_puerto

for i in grupo_puerto.index:
    puerto = grupo_puerto.loc[i]['puerto']
    empresa = grupo_puerto.loc[i]['empresa']
    importacion = grupo_puerto.loc[i]['importacion']
    ingrediente = grupo_puerto.loc[i]['ingrediente']

    fig = dibujar_inventario_puerto(df=puerto_df,
                                    empresa=empresa,
                                    puerto=puerto,
                                    ingrediente=ingrediente,
                                    importacion=importacion)
    fig.show()

In [ ]:
def dibujar_planta(df: pd.DataFrame, nombre_planta: str, ingrediente: str):

    temp = df[(df['planta'] == nombre_planta) &
              (df['ingrediente'] == ingrediente)]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=temp['fecha'],  # x-axis
        y=temp['inventario'],  # y-axis
        mode='lines',  # Connect data points with lines
        name='Inventario'  # Name in the legend
    )
    )

    fig.add_trace(go.Scatter(
        x=temp['fecha'],  # x-axis
        y=temp['capacidad'],  # y-axis
        mode='lines',  # Connect data points with lines
        name='Capacidad Almacenamiento',  # Name in the legend
        line=dict(dash='dash', color='blue')
    ))

    fig.add_trace(go.Bar(
        x=temp['fecha'],
        y=temp['llegada_bodega'],
        name='Despacho Bodega',
        marker_color='green',
        hovertext=temp['importaciones'])
    )

    fig.add_trace(go.Bar(
        x=temp['fecha'],
        y=temp['llegada_directa'],
        name='Despacho Directo',
        marker_color='orange',
        hovertext=temp['importaciones'])
    )

    fig.add_trace(go.Scatter(
        x=temp['fecha'],  # x-axis
        y=temp['safety_stock'],  # y-axis
        mode='lines',  # Connect data points with lines
        name='Safety Stock',  # Name in the legend
        line=dict(dash='dash', color='red')
    ))

    fig.add_trace(go.Bar(
        x=temp['fecha'],  # x-axis
        y=temp['llegada_programada'],  # y-axis
        name='Llegadas Programada',  # Name in the legend
        marker_color='gray'
    ))

    fig.add_trace(go.Bar(
        x=temp['fecha'],  # x-axis
        y=temp['backorder'],  # y-axis
        name='Backorder',  # Name in the legend
        marker_color='red'
    ))

    # Layout parameters
    fig.update_layout(
        title=f'Planta: {nombre_planta}: {ingrediente}',  # Title
        xaxis_title='Fecha',  # y-axis name
        yaxis_title='Kg',  # x-axis name
        xaxis_tickangle=0,  # Set the x-axis label angle
        showlegend=True,     # Display the legend
        barmode='stack',
        legend=dict(orientation='h',  x=0.5, xanchor='center'),
    )
    return fig

In [ ]:
for nombre, planta in problema.plantas.items(): 
    fig = dibujar_planta(df=reporte_plantas_df, nombre_planta=nombre, ingrediente='maiz')
    fig.show()